You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv() # Загружает переменные из .env файла в окружение

# Получение ключа из переменной окружения
api_key = os.getenv('OPENAI_API_KEY')

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.1-nano'
os.environ["OPENAI_API_BASE"] = 'https://api.proxyapi.ru/openai/v1'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
        "6. Translate all the resulting text to Russian "
            "so that it looked like written by native Russian speaker.\n"
    ),
    expected_output="A well-written blog post in Russian "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice. Make sure it is in Russian language only."),
    expected_output="A well-written blog post in Russian, in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Application of LLM to optimize taxi services"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c44531e0-69f9-4807-895c-0492dc01ab43                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: f4c4cc3e-f6ce-4c19-8edf-7b1a698cf800
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: f4c4cc3e-f6ce-4c19-8edf-7b1a698cf800
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Application of LLM to optimize taxi services.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


🤖 Agent: Content Planner
    Status: In Progress
└── 🧠 Thinking...

...

# Agent: Editor
## Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice. Make sure it is in Russian language only.


# Agent: Editor
## Final Answer: 
# Как крупные языковые модели (КЯМ) меняют транспорт: оптимизация такс услуг с помощью ИИ

## Введение: вызовы и возможности традиционного такси

Традиционная индустрия такси сталкивается с рядом серьезных вызовов, таких как устаревшие системы диспетчеризации, ограниченные ресурсы и неэффективная маршрутизация. Это приводит к высоким затратам, долгому времени ожидания и низкому уровню удовлетворенности клиентов. В современном мире быстрых технологий искусственный интеллект (ИИ), особенно крупные языковые модели (КЯМ), обещают произвести революцию в сфере мобильности.

Использование КЯМ дает возможность оптимизировать работу служб такси, повысить качество обслуживания и внедрить инновационные решения. Эта статья посвящена тому, как именно ИИ способствует развитию транспортных услуг, а также рассм

🚀 Crew: crew
├── 📋 Task: f4c4cc3e-f6ce-4c19-8edf-7b1a698cf800
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 24d73f69-ec58-470c-8832-28536a518233
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 8cecd004-2b18-439a-9d77-94727f54cdb9
       Status: Executing Task...
    └── 🤖 Agent: Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: f4c4cc3e-f6ce-4c19-8edf-7b1a698cf800
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 24d73f69-ec58-470c-8832-28536a518233
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 8cecd004-2b18-439a-9d77-94727f54cdb9
       Assigned to: Editor
       Status: ✅ Completed
    └── 🤖 Agent: Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8cecd004-2b18-439a-9d77-94727f54cdb9                                                                     │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c44531e0-69f9-4807-895c-0492dc01ab43                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result.raw)

# Как крупные языковые модели (КЯМ) меняют транспорт: оптимизация такс услуг с помощью ИИ

## Введение: вызовы и возможности традиционного такси

Традиционная индустрия такси сталкивается с рядом серьезных вызовов, таких как устаревшие системы диспетчеризации, ограниченные ресурсы и неэффективная маршрутизация. Это приводит к высоким затратам, долгому времени ожидания и низкому уровню удовлетворенности клиентов. В современном мире быстрых технологий искусственный интеллект (ИИ), особенно крупные языковые модели (КЯМ), обещают произвести революцию в сфере мобильности.

Использование КЯМ дает возможность оптимизировать работу служб такси, повысить качество обслуживания и внедрить инновационные решения. Эта статья посвящена тому, как именно ИИ способствует развитию транспортных услуг, а также рассмотрит основные перспективы и потенциальные сложности при его внедрении.

## Понимание мощи КЯМ и их ключевые возможности

Крупные языковые модели (например, GPT-4, ChatGPT) обладают способностью к пониманию и генерации человеческого языка, что позволяет им взаимодействовать с пользователями максимально естественно. Их функции включают автоматизированное ведение диалогов, обработку клиентских запросов, подготовку отчетов и поддержку принимаемых решений.

В транспортной сфере КЯМ помогают обрабатывать большие объемы данных в реальном времени — от погодных условий и дорожной ситуации до текущего спроса на услуги такси. Это позволяет операторам принимать более обоснованные и своевременные решения, увеличивая эффективность работы парков и повышая безопасность пассажиров и водителей. В результате КЯМ становятся неотъемлемой частью современных решений в области ride-hailing и мобильной логистики.

## Как КЯМ оптимизируют услуги такси

### Реальное время: маршруты и диспетчеризация

Объединение КЯМ с системами GPS и дорожной аналитикой позволяет значительно повысить точность и оперативность маршрутизации. Системы на базе ИИ могут автоматически переадресовывать водителей в зоны с повышенным спросом, предлагать наиболее короткие маршруты или учитывать текущие дорожные условия. В результате ожидание клиента сокращается, а расход топлива — снижается, что помогает повысить прибыльность и экологичность перевозок.

### Улучшение взаимодействия с клиентами

Технологии на базе КЯМ активно внедряются в чатботы и автоматические системы поддержки. Они позволяют обрабатывать заявки на заказ, получать отзывы и решать типичные вопросы пользователей моментально и без участия человека. Благодаря использованию естественного языка, такие системы создают ощущение персонализированного подхода, повышают уровень доверия и удовлетворенности клиентов, а также снижает нагрузку на операторов колл-центров.

### Прогнозирование спроса и динамическое ценообразование

КЯМ помогают прогнозировать пики спроса, анализируя исторические данные и текущие параметры рынка. Так, они позволяют вовремя корректировать тарифы, внедряя динамическое ценообразование, которое балансирует спрос и предложение. Это важно для оптимизации доходов в периоды повышенной активности или экстремальных ситуаций, а также обеспечивает более стабильную работу сервиса.

### Поддержка и обучение водителей

Водителям предоставляются актуальные советы по маршрутам, предупреждения о текущей дорожной обстановке и напоминания о правилах безопасности. Также системы, основанные на КЯМ, анализируют отчеты и отзывы водителей, что помогает своевременно выявлять проблемы и повышать качество предоставляемых услуг.

### Безопасность и соответствие нормативам

КЯМ могут анализировать переписку и коммуникацию между пассажирами и водителями на предмет потенциальных угроз или неправомерных действий. Автоматически формируя отчеты и рекомендации, такие системы способствуют соблюдению нормативных требований и повышению уровня безопасности, предотвращая возможные инциденты.

## Кто уже внедряет КЯМ и их достижения

Компании-лидеры в области ride-hailing активно используют ИИ для повышения эффективности своих сервисов. Uber использует системы на базе ИИ для оптимизации маршрутов и поддержки клиентов, Lyft — автоматизированные чатботы и анализ спроса, а Didi Chuxing делает акцент на аналитике безопасности и автоматизированной коммуникации.

Недавние пилотные проекты показывают существенные результаты: сокращение времени ожидания, повышение точности маршрутов и улучшение показателей безопасности. Инвестиции в ИИ-технологии продолжают расти, что свидетельствует о стратегической важности внедрения КЯМ в будущее транспортных систем.

## Перспективы и вызовы: что ждет индустрию?

В будущем нас ждут полностью автономные такси, интеграция с мультимодальными транспортными системами и смещение акцента на экологичность и безопасность. Однако внедрение ИИ требует решения таких вопросов, как защита данных, этичность алгоритмов и предотвращение предвзятости.

Компаниям важно активнее развивать компетенции в области ИИ, разрабатывать безопасные и прозрачные модели, а также участвовать в нормативных дискуссиях. Только так можно добиться гармоничного сочетания инноваций и стабильности.

## Заключение: путь к умному и безопасному транспорту

Использование КЯМ открывает новые горизонты для повышения эффективности, безопасности и удобства пассажиров. Эти модели помогают оптимизировать маршруты, предсказывать спрос, автоматизировать коммуникации и снижать риски. В будущем развитие ИИ обещает сделать транспорт более адаптивным, экологичным и безопасным.

Для отрасли важно активно внедрять инновационные технологии, оставаясь конкурентоспособными на рынке и отвечая требованиям современного города будущего. Руководители транспортных компаний и разработчики должны рассматривать ИИ не только как средство повышения прибыли, но и как инструмент для обеспечения высокого уровня сервиса.

## Итог: инновации на каждом шагу

Крупные языковые модели уже сегодня вносят заметные изменения в транспортную отрасль, улучшая маршрутизацию, взаимодействие с клиентами и процессы безопасности. Они помогают службам такси становиться более эффективными, а пассажирам — получать более комфортные и надежные услуги. Те, кто уже инвестируют в развитие ИИ, создают будущее умной мобильности.

> Время инноваций — именно сейчас. Компании, использующие потенциал ИИ, смогут не только укрепить свои позиции на рынке, но и обеспечить своим клиентам безопасность, комфорт и высокий уровень сервиса в эпоху быстрых перемен.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Выбор между Kotlin и Python для разработки систем ИИ"
result = crew.kickoff(inputs={"topic": topic})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c44531e0-69f9-4807-895c-0492dc01ab43                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: f4c4cc3e-f6ce-4c19-8edf-7b1a698cf800
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: f4c4cc3e-f6ce-4c19-8edf-7b1a698cf800
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Выбор между Kotlin и Python для разработки систем ИИ.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
**Comprehensive Content Plan for Blog Article: "Выбор между Kotlin и Python для разработки систем ИИ"**

---

### 1. Prioritization of Latest Trends, Key Players, and Noteworthy News

**Latest Trends:**
- **Python's Dominance in AI Development:** Python continues to be the most popular language for AI and machine learning, primarily due to extensive libraries (TensorFlow, PyTorch, scikit-learn) and ease of use.
- **Emerging Use of Kotlin:** Kotlin is gaining traction in mobile and backend development, and recently, efforts have been made to support AI developme

🚀 Crew: crew
└── 📋 Task: f4c4cc3e-f6ce-4c19-8edf-7b1a698cf800
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: ✅ Completed

...

# Agent: Editor
## Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice. Make sure it is in Russian language only.


# Agent: Editor
## Final Answer: 
# Выбор между Kotlin и Python для разработки систем ИИ: что лучше для вашего проекта?

В современном мире искусственный интеллект занимает все более важное место в технологическом прогрессе. Разработка систем ИИ требует не только глубоких знаний в области алгоритмов и моделей, но и правильного выбора инструментов — в первую очередь, языка программирования. Среди множества вариантов особенно выделяются Python и Kotlin — два современных языка, каждый из которых имеет свои преимущества и ограничения. В этой статье мы внимательно рассмотрим их особенности и поможем вам определить, какой язык лучше подходит для разработки систем искусственного интеллекта именно в вашем случае.

## Kotlin в мире ИИ: новые возможности и перспективы

Kotlin уже давно зарекомендовал себя как мощный мультипарадигменный яз

🚀 Crew: crew
├── 📋 Task: f4c4cc3e-f6ce-4c19-8edf-7b1a698cf800
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 24d73f69-ec58-470c-8832-28536a518233
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 8cecd004-2b18-439a-9d77-94727f54cdb9
       Status: Executing Task...
    └── 🤖 Agent: Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: f4c4cc3e-f6ce-4c19-8edf-7b1a698cf800
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: 24d73f69-ec58-470c-8832-28536a518233
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 8cecd004-2b18-439a-9d77-94727f54cdb9
       Assigned to: Editor
       Status: ✅ Completed
    └── 🤖 Agent: Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8cecd004-2b18-439a-9d77-94727f54cdb9                                                                     │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c44531e0-69f9-4807-895c-0492dc01ab43                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [15]:
Markdown(result.raw)

# Выбор между Kotlin и Python для разработки систем ИИ: что лучше для вашего проекта?

В современном мире искусственный интеллект занимает все более важное место в технологическом прогрессе. Разработка систем ИИ требует не только глубоких знаний в области алгоритмов и моделей, но и правильного выбора инструментов — в первую очередь, языка программирования. Среди множества вариантов особенно выделяются Python и Kotlin — два современных языка, каждый из которых имеет свои преимущества и ограничения. В этой статье мы внимательно рассмотрим их особенности и поможем вам определить, какой язык лучше подходит для разработки систем искусственного интеллекта именно в вашем случае.

## Kotlin в мире ИИ: новые возможности и перспективы

Kotlin уже давно зарекомендовал себя как мощный мультипарадигменный язык, особенно популярный среди разработчиков мобильных приложений благодаря тесной интеграции с платформой Android. В последние годы Kotlin начал активно расширять свои возможности и в сфере ИИ. Благодаря поддержке Kotlin Native и появлению таких библиотек, как KotlinDL, возникают новые шансы использовать этот язык в задачах машинного обучения и анализа данных.

Несмотря на то, что экосистема для искусственного интеллекта в Kotlin пока что развивается относительно недавно, она привлекает внимание индустрии благодаря своей переносимости и возможности интеграции с существующими Java-библиотеками. В целом, Kotlin позиционируется как хороший выбор для внедрения ИИ в мобильные и enterprise-системы, особенно если проект уже использует платформу JVM. Однако стоит помнить, что сообщество разработчиков и инфраструктура поддержки еще не достигли уровня Python.

## Почему Python остаётся лидером в области ИИ

Очевидно, что Python занимает доминирующую позицию в разработке систем ИИ и машинного обучения. Это обусловлено простотой языка, гибкостью и, самое главное, богатством специализированных библиотек. TensorFlow, PyTorch, scikit-learn — только несколько из многочисленных инструментов, предоставляющих широкие возможности для обучения и внедрения моделей ИИ. Кроме того, огромное сообщество разработчиков и активные исследования делают Python предпочтительным выбором для научной деятельности и быстрого прототипирования.

Практика показывает, что большинство крупных технологических компаний, таких как Google, Facebook и OpenAI, используют именно Python для своих AI-решений. Это создает сильную индустриальную поддержку и продолжает стимулировать развитие новых библиотек и инструментов, что делает Python надежным и перспективным языком для будущих проектов искусственного интеллекта.

## Сравнение Kotlin и Python: технические и практические аспекты

Когда речь заходит о сравнении Python и Kotlin в контексте AI, важно учитывать несколько ключевых аспектов.

Во-первых, производительность: Kotlin, работая на JVM, демонстрирует высокую скорость выполнения и хорошую масштабируемость для крупных систем. В то время как Python является интерпретируемым языком, что иногда может накладывать ограничения при масштабных задачах, особенно если речь идёт о высоких требованиях к скорости.

Во-вторых, развитие и простота использования. Python славится своими легкими синтаксисом и быстротой написания кода, что очень важно для исследовательских и прототипных работ. Kotlin в этом отношении чуть менее гибок, но его преимущества проявляются в проектах, где важна интеграция с существующими платформами и стабильность.

В-третьих, экосистема и поддержка библиотек. Python богат многочисленными библиотеками для машинного обучения и обработки данных. Kotlin, несмотря на небольшое, но растущее сообщество, пока что не может конкурировать по количеству готовых решений, особенно в области глубинного обучения. Однако это компенсируется возможностью использования JVM-библиотек, что обеспечивает некоторую гибкость.

## Практические советы: когда выбрать Kotlin, а когда — Python?

Если ваш проект ориентирован на мобильные приложения, и вам требуется интеграция с Android, или вы разрабатываете enterprise-решения на базе JVM, Kotlin станет более логичным выбором. Особенно это касается внедрения AI в системы, где важна мультиплатформенность и высокая производительность.

С другой стороны, для быстрой разработки прототипов, проведения научных исследований или работы с большими наборами данных предпочтительнее использовать Python. Благодаря богатству библиотек и простоте языка, он позволяет быстрее реализовать идеи и получить рабочие решения.

Также возможен гибридный подход — использование Kotlin для части системы, связанной с мобильными или корпоративными приложениями, и Python для создания и обучения моделей, которые затем интегрируются в Kotlin-приложение при помощи API или микросервисов.

## Будущее и тренды: что ожидать от языков в сфере ИИ?

Рынок развивается в сторону многоплатформенных решений и автоматизации процессов. Kotlin показывает потенциал для расширения своей роли в корпоративных системах и мобильных приложениях с элементами ИИ благодаря появлению новых библиотек и поддержке мультиплатформенной разработки.

Python же продолжит оставаться основным инструментом для исследователей и тех, кто ищет скорость прототипирования и широкую поддержку технологий искусственного интеллекта. В перспективе можно ожидать более глубокой интеграции этих языков, что позволит создавать гибридные системы современного уровня.

## Итог: какой язык выбрать для разработки систем ИИ?

Подводя итог, можно сказать, что выбор между Kotlin и Python зависит от специфики вашего проекта и поставленных задач. Для внедрения решений в мобильной сфере, а также в корпоративных системах с необходимостью интеграции AI лучше подойдет Kotlin, особенно с учетом его перспектив развития и поддержки со стороны JetBrains. В то же время, для быстрого прототипирования, научных исследований и проектов с широким сообществом предпочтительнее использовать Python.

Не бойтесь экспериментировать, сочетать подходы и искать новые решения — технологии постоянно развиваются, и возможность гибко адаптироваться к новым условиям является залогом успешного внедрения искусственного интеллекта.

---

Если вы желаете укрепить свои знания в области AI и выбрать наиболее подходящий язык для разработки систем ИИ, начните с анализа требований вашего проекта и попробуйте оба варианта. Постоянная практика и изучение новых инструментов позволят вам принимать более обоснованные решения и реализовать наиболее эффективные решения в области искусственного интеллекта.

---

**Обратите внимание:** статья предназначена для того, чтобы помочь вам ориентироваться в текущем состоянии технологий и трендах индустрии. Удачи в ваших проектах!

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).